# Getting started with Layer

[![Open in Layer](https://app.layer.ai/assets/badge.svg)](https://development.layer.co/layer/titanic) [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/layerai/examples/blob/main/titanic/Getting_Started_With_Layer.ipynb) [![Layer Examples Github](https://badgen.net/badge/icon/github?icon=github&label)](https://github.com/layerai/examples/tree/main/titanic)

In this quick walkthrough, we will train a machine learning model to predict the survivors of the Titanic disaster and deploy it for real-time inference using Layer.


## Installation


In [ ]:
!pip install --upgrade layer-sdk -q

In [12]:
from layer.decorators import dataset, model,resources
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import numpy as np
import layer

## Register and Login

To start using Layer, you have to register and login. Run the following cell, click the link, register and paste the code in the input

In [ ]:
layer.login()

## Inititialize Your First Layer Project

It's time to create your first Layer Project. 

In [ ]:
layer.init("titanic")

## Build Passengers Dataset

Let's start building our data to train our model. We will be using the Kaggle Titanic Dataset which consists two datasets:

- train.csv
- test.csv

Let's clone the [Layer Titanic Project](https://github.com/layerai/examples/tree/main/titanic) repo which has these datasets.

In [ ]:
!git clone https://github.com/layerai/examples
!mv ./examples/titanic/* ./

Ok, we have our data ready in the `./data` folder. Now we are going to be building the Passengers dataset and register to Layer.

In [ ]:
def clean_gender(sex):
    result = 0
    if sex == "female":
        result = 0
    elif sex == "male":
        result = 1
    return result


def clean_age(data):
    age = data[0]
    pclass = data[1]
    if pd.isnull(age):
        if pclass == 1:
            return 37
        elif pclass == 2:
            return 29
        else:
            return 24
    else:
        return age  

@dataset("passengers")
@resources(path="./data")
def build_passengers_dataset():
  train_df = pd.read_csv("data/train.csv")
  test_df = pd.read_csv("data/test.csv")
  df = train_df.append(test_df)

  df['Sex'] = df['Sex'].apply(clean_gender)
  df['Age'] = df[['Age', 'Pclass']].apply(clean_age, axis=1)
  df = df.drop(["PassengerId", "Name", "Cabin", "Ticket", "Embarked"], axis=1)

  return df

In [ ]:
#build_passengers_dataset()

layer.run([build_passengers_dataset])

## Train Survival Model

In [ ]:
@model(name='survival_model')
def train():
    test_size = 0.3
    random_state = 42
    n_estimators = 100

    layer.log({"test_size": test_size, 
               "random_state" : random_state,
               "n_estimators": n_estimators})

    df = layer.get_dataset("passengers").to_pandas()
    X = df.drop(["Survived"], axis=1)
    y = df["Survived"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    random_forest = RandomForestClassifier(n_estimators=n_estimators)
    random_forest.fit(X_train, y_train)
    y_pred = random_forest.predict(X_test)
    return random_forest

In [ ]:
#train()

layer.run([train])

## Results

After you train your model, you can see all your datasets and model experiments here in the Layer interface

https://app.layer.ai/

Or you can re-use one of the entities you have created.

### Fetching an ML model from Layer

After you build the Layer Project, you can fetch your trained models from Layer with simple calling `layer.get_model("MODEL_NAME")`

If you [make your project public](https://docs.app.layer.ai/docs/projects/create-project), anyone even without a Layer account can fetch your models for their own use.

In [ ]:
survival_model = layer.get_model("survival_model")
survival_model.metrics

### Fetching a dataset from Layer

Just like the models, you can fetch a dataset that you have built under a Layer Project. Anyone can fetch and use your dataset if your project is public

In [ ]:
df = layer.get_dataset("passengers").to_pandas()
df.head()

Now, let's predict the survival probability of a passenger with the model and dataset we have trained and registered to Layer.

In [ ]:
passenger = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]
survival_propability = train.predict_proba(passenger.sample())[0][1]

print(f"Survival Probaility: {survival_propability:.2%}")

## Where to go from here?

Now that you have created first Layer Project, you can:

- Join our [Slack Community ](https://bit.ly/layercommunityslack)
- Visit [Layer Examples Repo](https://github.com/layerai/examples) for more examples
- Browse [Trending Layer Projects](https://layer.ai) on our mainpage
- Check out [Layer Documentation](https://docs.layer.ai) to learn more